# Welcome to the API PT2!

## Data Edition

#### Goal - Filter down to the pit density we want and plot it

### Step 1. Imports

In [5]:
# imports
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
from snowexsql.api import PointMeasurements, LayerMeasurements

### Step 2. Find the pits in the Boise River Basin

In [4]:
# Find site names we can use
print(LayerMeasurements().all_site_names)

[('Cameron Pass',), ('Sagehen Creek',), ('Fraser Experimental Forest',), ('Mammoth Lakes',), ('Niwot Ridge',), ('Boise River Basin',), ('Little Cottonwood Canyon',), ('East River',), ('American River Basin',), ('Senator Beck',), ('Jemez River',), ('Grand Mesa',)]


In [6]:
# Get the first 1000 measurements from the Boise River Basin Site
df = LayerMeasurements.from_filter(
    type="density",
    site_name="Boise River Basin",
    limit=1000
)

# Explore the pits so we can find an interesting site
df.loc[:, ["site_id", "geom"]].drop_duplicates().explore()

### Step 3. Pick a point of interest

In [17]:
# We noticed there are a lot of pits (timeseries pits) for Banner Open
# Filter down to ONE timeseries
site_id = "Banner Open"
df = LayerMeasurements.from_filter(
    type="density",
    site_id=site_id
).set_crs("epsg:26911")

df.loc[:, ["site_id", "geom"]].drop_duplicates().explore()


In [8]:
# Get the mean of each date sampled
df["value"] = df["value"].astype(float)
df.set_index("date", inplace=True)
mean_values = df.groupby(df.index).mean()
mean_values.head()

,id,latitude,longitude,northing,easting,utm_zone,depth,bottom_depth,value
date,,,,,,,,,
2019-12-18,22609.5,44.30464,-115.23603,4.907222e+06,640699.666121,11.0,26.000000,16.000000,279.000000
2020-01-09,22670.0,44.30463,-115.23601,4.907221e+06,640701.285289,11.0,51.000000,41.000000,156.777778
2020-01-23,22746.0,44.30461,-115.23598,4.907219e+06,640703.725988,11.0,61.727273,51.727273,254.166667
2020-01-30,22830.5,44.30461,-115.23598,4.907219e+06,640703.725988,11.0,72.000000,62.000000,236.000000
2020-02-06,22910.0,44.30458,-115.23594,4.907216e+06,640706.988222,11.0,72.000000,62.000000,254.141026


### Notes on this mean

Taking this `mean` as bulk density **could be flawed** if layers are overlapping or layers vary in thickness

In [12]:
# !pip install plotly
import plotly.express as px


In [18]:
# Plot the timeseries of mean density
fig = px.line(
    mean_values, x=mean_values.index, y='value',
    title=f'Mean Density - {site_id}',
    labels={'value': 'Density', 'date': 'Date'}
)

fig.update_layout(
    template='plotly_dark'
)

# Show the plot
fig.show()

# alternative matplotlib code
# mean_values["value"].plot()
# plt.title('Mean Density by Date')
# plt.xlabel('Date')
# plt.ylabel('Mean Density')
# plt.gcf().autofmt_xdate()
# plt.show()

In [19]:
# Show more detail by using a box plot
fig = px.box(df, x='date', y='value', notched=True, title='Pit Density by Date')
fig.update_traces(hoverinfo='y+name')
fig.update_layout(template='plotly_dark')

# alternative matplotlib code
# df.boxplot(by='date', column='value')
# plt.title('Density Distribution for Banner by Date')
# plt.suptitle('')  # Suppress the automatic title
# plt.xlabel('Date')
# plt.ylabel('Value')
# plt.show()